In [1]:
%config Completer.use_jedi = False
from ModelEngine import ModelEngine
neo = ModelEngine('normal_neonate.json').io

normal neonate model loaded and initialized correctly.


In [10]:
neo.calculate(1800)
ALL = neo.model.components['ALL']
OUT = neo.model.components['OUT']

Calculating model run of 1800 sec. in 3600000 steps.
Ready in 46.639 sec. Average model step in 0.013 ms.


In [11]:
ALL.gas_partial_pressures

[142.24987423537797,
 6.246772520121611,
 0.27159880522267876,
 530.2287674959746,
 44.86356120129649]

In [12]:
OUT.gas_partial_pressures

[156.05152161871857,
 6.852859183256398,
 0.297950399272019,
 581.6736669788061,
 7.52400020146762]